In [1]:
import numpy as np
from itertools import product
import math
import random
import copy

#### Test Metropolis

In [73]:
class Metropolis:
    def __init__(self, H):
        self.H = H
        # Possible values the spins can take
        self.spin_values = [-1,1]
        # Calculate the shape of the matrix and save useful variables
        self.num_spins = 1
        
    def metropolis_step(self):
        # Randomly select a spin (now it can only select 0)
        spin_to_flip = np.random.randint(self.num_spins)

        delta_energy = 2*(self.H[spin_to_flip]*self.sigma[spin_to_flip])

        p = math.exp(-delta_energy)
        r = random.random()

        #print(delta_energy, r, p)
        if delta_energy <= 0 or r<=p:
            self.sigma[spin_to_flip] = -self.sigma[spin_to_flip]

    def metropolis_simulation(self, max_steps, burn_in_len=0):
        # Generate a random initial spin configuration
        self.sigma = random.choices(self.spin_values, k=self.num_spins)
        
        # Create a dictionary to count the frequencies
        samples_dict = {}
        for i in range(0, max_steps):
            # Take a metropolis step
            self.metropolis_step()
            
            # Save the state if the burn-in phase has passed
            if i >= burn_in_len:
                
                if not tuple(self.sigma) in samples_dict: # If the state is new, create an entry
                    samples_dict[tuple(self.sigma)] = 1
                else: # If the state already appeared, add one
                    samples_dict[tuple(self.sigma)] += 1

        # Normalize the probability
        sum_states = sum(samples_dict.values())
        normalized_dict = {k: v / sum_states for k, v in samples_dict.items()}
                                                 
        return normalized_dict

    def get_statistics(self, states_dict):
        total =  np.zeros(len(self.H))
        # Calculate the expectation
        for sigma, prob in states_dict.items():
            total += np.dot(sigma,prob)
            
        return total

    def test(self, simulation_len, burn_in_len):
        print('Initial H', self.H)
        print('Magnetization', np.tanh(self.H))
        
        samples_dict = self.metropolis_simulation(simulation_len, burn_in_len)
        
        print(samples_dict)

        # Calculate the data expectations        
        model_magnetization = self.get_statistics(samples_dict)
        
        
        print('Metropolis magnetization', model_magnetization)
        print('Diff', model_magnetization - np.tanh(self.H))

In [74]:
H = [0.70]
metropolis = Metropolis(H)

In [77]:
metropolis.test(simulation_len=1000, burn_in_len=100)

Initial H [0.7]
Magnetization [0.60436778]
{(1,): 0.8177777777777778, (-1,): 0.18222222222222223}
Metropolis magnetization [0.63555556]
Diff [0.03118778]


In [72]:
# Should be around 0 if random
acum = 0
for i in range(0,10000):   
    v = random.choices([-1, 1], k=1)
    acum += v[0]
acum

100

#### Only H

In [26]:
class SimpleIsingModel:
    def __init__(self, data, lr=0.0005):
        self.lr = lr
        
        # Possible values the spins can take
        self.spin_values = [-1,1]
        
        # Create a ndarray from the data
        self.data_activations_matrix = np.array(data)
        # Calculate the shape of the matrix and save useful variables
        self.num_spins = self.data_activations_matrix.shape[1]
        self.num_samples_data = self.data_activations_matrix.shape[0]
        
        # Calculate the true data expectations
        # The mean
        self.data_mean = np.mean(self.data_activations_matrix, axis=0)
        
                
        # Calculate the initial H vector, each entry proportional to its probability (according to Hinton)
        #self.H = np.sum(self.data_activations_matrix, axis=0)/ float(self.num_samples_data)
        # Try with random H
        self.H = np.random.normal(loc=0.0, scale=.01, size=self.num_spins)
        
        
    # Train the exact model
    def train(self, max_epochs = 500):
        
        totalParamVariation = math.inf
        stopCondition = 0.0000005
        
        epoch = 1
        
        while totalParamVariation > stopCondition and epoch < max_epochs:
            # Calculate P(sigma) for every possible combination in the model

            prob_dict = {}
            for sigma in product(self.spin_values, repeat=self.num_spins):
                prob_dict[sigma] = math.exp(-self.calculate_energy(sigma))

            # Calculate the partition function
            Z = sum(prob_dict.values())

            # Normalize dividing by the partition function
            for sigma in prob_dict.keys():
                prob_dict[sigma] = prob_dict[sigma]/Z

            model_mean = np.zeros(self.H.shape)
            for sigma in product(self.spin_values, repeat=self.num_spins):
                # Calculate the expectation of the averages
                for i in range(0, self.num_spins):
                    model_mean[i] += sigma[i]*prob_dict[sigma]


            # Calculate the step size for every H_{i}
            stepH = self.lr * (self.data_mean - model_mean)
            
            oldH = copy.deepcopy(self.H)
            # Take the step
            self.H = self.H + stepH
            
            # Calculate the variation of J for the termination condition
            totalParamVariation = np.sum(np.absolute(self.H - oldH))

            if epoch%100 == 0:
                print('Epoch', epoch, 'TotalParamVariation', round(totalParamVariation, 8))
                print(model_mean, 'Average Model')
                print(self.data_mean, 'Average Data')
                print(self.data_mean - model_mean, 'Diff')
                print()
            
            epoch += 1

        print(self.data_mean, 'Spins average')
        print(self.H, 'H')
    
    def metropolis_step(self):
        # Randomly select a spin (now it can only select 0)
        spin_to_flip = np.random.randint(self.num_spins)

        delta_energy = 2*(self.H[spin_to_flip]*self.sigma[spin_to_flip])

        p = math.exp(-delta_energy)
        r = random.random()

        #print(delta_energy, r, p)
        if delta_energy <= 0 or r<=p:
            self.sigma[spin_to_flip] = -self.sigma[spin_to_flip]

    def metropolis_simulation(self, max_steps, burn_in_len=0):
        # Generate a random initial spin configuration
        self.sigma = random.choices(self.spin_values, k=self.num_spins)
        
        # Create a dictionary to count the frequencies
        samples_dict = {}
        for i in range(0, max_steps):
            # Take a metropolis step
            self.metropolis_step()
            
            # Save the state if the burn-in phase has passed
            if i >= burn_in_len:
                
                if not tuple(self.sigma) in samples_dict: # If the state is new, create an entry
                    samples_dict[tuple(self.sigma)] = 1
                else: # If the state already appeared, add one
                    samples_dict[tuple(self.sigma)] += 1

        # Normalize the probability
        sum_states = sum(samples_dict.values())
        normalized_dict = {k: v / sum_states for k, v in samples_dict.items()}
                                                 
        return normalized_dict

    def get_statistics(self, states_dict):
        total =  np.zeros(len(self.H))
        # Calculate the expectation
        for sigma, prob in states_dict.items():
            total += np.dot(sigma,prob)
            
        return total
    
    def train_metropolis(self, max_epochs = 300, simulation_len = 100, burn_in_len=0):
        random.seed(3)
        totalParamVariation = math.inf
        stopCondition = 0.0000000005
        
        epoch = 1
        
        while totalParamVariation > stopCondition and epoch < max_epochs:

            samples_dict = self.metropolis_simulation(simulation_len, burn_in_len)
        
            # Calculate the data expectations        
            model_magnetization = self.get_statistics(samples_dict)
            
            # Calculate the step size for every H_{i}
            stepH = self.lr * (self.data_mean - model_magnetization)
            # Take the step
            oldH = copy.deepcopy(self.H)
            self.H = self.H + stepH
            
            # Calculate the variation of H for the stop condition
            totalParamVariation = np.sum(np.absolute(self.H - oldH))
            
            if epoch > 300:
                print('Epoch', epoch, 'TotalParamVariation', round(totalParamVariation, 8))
                print(model_magnetization, 'Average Model')
                print(self.data_mean, 'Average Data')
                print(self.data_mean - model_magnetization, 'Diff')
                print()

            epoch +=1
        
        print(self.data_mean, 'Spins average')
        print(self.H, 'H')


In [32]:
activations_list = [[1], [1], [-1], [-1], [1], [1], [1], [1]]
s_ising = SimpleIsingModel(activations_list, lr=0.01)


In [33]:
s_ising.train_metropolis( max_epochs = 3000, simulation_len = 100, burn_in_len=10)

Epoch 301 TotalParamVariation 0.00211111
[0.28888889] Average Model
[0.5] Average Data
[0.21111111] Diff

Epoch 302 TotalParamVariation 0.00055556
[0.44444444] Average Model
[0.5] Average Data
[0.05555556] Diff

Epoch 303 TotalParamVariation 0.00033333
[0.53333333] Average Model
[0.5] Average Data
[-0.03333333] Diff

Epoch 304 TotalParamVariation 0.00077778
[0.42222222] Average Model
[0.5] Average Data
[0.07777778] Diff

Epoch 305 TotalParamVariation 0.00055556
[0.44444444] Average Model
[0.5] Average Data
[0.05555556] Diff

Epoch 306 TotalParamVariation 0.00055556
[0.55555556] Average Model
[0.5] Average Data
[-0.05555556] Diff

Epoch 307 TotalParamVariation 0.00011111
[0.51111111] Average Model
[0.5] Average Data
[-0.01111111] Diff

Epoch 308 TotalParamVariation 0.00011111
[0.48888889] Average Model
[0.5] Average Data
[0.01111111] Diff

Epoch 309 TotalParamVariation 0.00055556
[0.44444444] Average Model
[0.5] Average Data
[0.05555556] Diff

Epoch 310 TotalParamVariation 0.00144444
[0

Epoch 417 TotalParamVariation 0.001
[0.4] Average Model
[0.5] Average Data
[0.1] Diff

Epoch 418 TotalParamVariation 0.001
[0.6] Average Model
[0.5] Average Data
[-0.1] Diff

Epoch 419 TotalParamVariation 0.00033333
[0.53333333] Average Model
[0.5] Average Data
[-0.03333333] Diff

Epoch 420 TotalParamVariation 0.00033333
[0.46666667] Average Model
[0.5] Average Data
[0.03333333] Diff

Epoch 421 TotalParamVariation 0.00011111
[0.48888889] Average Model
[0.5] Average Data
[0.01111111] Diff

Epoch 422 TotalParamVariation 0.00055556
[0.44444444] Average Model
[0.5] Average Data
[0.05555556] Diff

Epoch 423 TotalParamVariation 0.00077778
[0.42222222] Average Model
[0.5] Average Data
[0.07777778] Diff

Epoch 424 TotalParamVariation 0.00011111
[0.51111111] Average Model
[0.5] Average Data
[-0.01111111] Diff

Epoch 425 TotalParamVariation 0.00077778
[0.42222222] Average Model
[0.5] Average Data
[0.07777778] Diff

Epoch 426 TotalParamVariation 0.00033333
[0.53333333] Average Model
[0.5] Average

Epoch 534 TotalParamVariation 0.00033333
[0.46666667] Average Model
[0.5] Average Data
[0.03333333] Diff

Epoch 535 TotalParamVariation 0.00011111
[0.48888889] Average Model
[0.5] Average Data
[0.01111111] Diff

Epoch 536 TotalParamVariation 0.00033333
[0.53333333] Average Model
[0.5] Average Data
[-0.03333333] Diff

Epoch 537 TotalParamVariation 0.00033333
[0.46666667] Average Model
[0.5] Average Data
[0.03333333] Diff

Epoch 538 TotalParamVariation 0.00077778
[0.42222222] Average Model
[0.5] Average Data
[0.07777778] Diff

Epoch 539 TotalParamVariation 0.00011111
[0.51111111] Average Model
[0.5] Average Data
[-0.01111111] Diff

Epoch 540 TotalParamVariation 0.001
[0.4] Average Model
[0.5] Average Data
[0.1] Diff

Epoch 541 TotalParamVariation 0.00033333
[0.53333333] Average Model
[0.5] Average Data
[-0.03333333] Diff

Epoch 542 TotalParamVariation 0.00077778
[0.42222222] Average Model
[0.5] Average Data
[0.07777778] Diff

Epoch 543 TotalParamVariation 0.00033333
[0.53333333] Average 

Epoch 666 TotalParamVariation 0.00033333
[0.46666667] Average Model
[0.5] Average Data
[0.03333333] Diff

Epoch 667 TotalParamVariation 0.00077778
[0.42222222] Average Model
[0.5] Average Data
[0.07777778] Diff

Epoch 668 TotalParamVariation 0.00033333
[0.46666667] Average Model
[0.5] Average Data
[0.03333333] Diff

Epoch 669 TotalParamVariation 0.00055556
[0.44444444] Average Model
[0.5] Average Data
[0.05555556] Diff

Epoch 670 TotalParamVariation 0.00055556
[0.55555556] Average Model
[0.5] Average Data
[-0.05555556] Diff

Epoch 671 TotalParamVariation 0.00011111
[0.51111111] Average Model
[0.5] Average Data
[-0.01111111] Diff

Epoch 672 TotalParamVariation 0.00033333
[0.46666667] Average Model
[0.5] Average Data
[0.03333333] Diff

Epoch 673 TotalParamVariation 0.00033333
[0.53333333] Average Model
[0.5] Average Data
[-0.03333333] Diff

Epoch 674 TotalParamVariation 0.00011111
[0.48888889] Average Model
[0.5] Average Data
[0.01111111] Diff

Epoch 675 TotalParamVariation 0.00055556
[0

[0.44444444] Average Model
[0.5] Average Data
[0.05555556] Diff

Epoch 764 TotalParamVariation 0.00055556
[0.44444444] Average Model
[0.5] Average Data
[0.05555556] Diff

Epoch 765 TotalParamVariation 0.001
[0.6] Average Model
[0.5] Average Data
[-0.1] Diff

Epoch 766 TotalParamVariation 0.00055556
[0.55555556] Average Model
[0.5] Average Data
[-0.05555556] Diff

Epoch 767 TotalParamVariation 0.00033333
[0.53333333] Average Model
[0.5] Average Data
[-0.03333333] Diff

Epoch 768 TotalParamVariation 0.00033333
[0.46666667] Average Model
[0.5] Average Data
[0.03333333] Diff

Epoch 769 TotalParamVariation 0.00011111
[0.51111111] Average Model
[0.5] Average Data
[-0.01111111] Diff

Epoch 770 TotalParamVariation 0.00033333
[0.53333333] Average Model
[0.5] Average Data
[-0.03333333] Diff

Epoch 771 TotalParamVariation 0.00122222
[0.37777778] Average Model
[0.5] Average Data
[0.12222222] Diff

Epoch 772 TotalParamVariation 0.00055556
[0.55555556] Average Model
[0.5] Average Data
[-0.05555556] 

...

[0.5] Average Data
[-0.01111111] Diff

Epoch 2474 TotalParamVariation 0.00055556
[0.44444444] Average Model
[0.5] Average Data
[0.05555556] Diff

Epoch 2475 TotalParamVariation 0.00011111
[0.51111111] Average Model
[0.5] Average Data
[-0.01111111] Diff

Epoch 2476 TotalParamVariation 0.00055556
[0.44444444] Average Model
[0.5] Average Data
[0.05555556] Diff

Epoch 2477 TotalParamVariation 0.00077778
[0.42222222] Average Model
[0.5] Average Data
[0.07777778] Diff

Epoch 2478 TotalParamVariation 0.00055556
[0.55555556] Average Model
[0.5] Average Data
[-0.05555556] Diff

Epoch 2479 TotalParamVariation 0.00011111
[0.51111111] Average Model
[0.5] Average Data
[-0.01111111] Diff

Epoch 2480 TotalParamVariation 0.00011111
[0.48888889] Average Model
[0.5] Average Data
[0.01111111] Diff

Epoch 2481 TotalParamVariation 0.00077778
[0.57777778] Average Model
[0.5] Average Data
[-0.07777778] Diff

Epoch 2482 TotalParamVariation 0.00055556
[0.55555556] Average Model
[0.5] Average Data
[-0.05555556]

Epoch 2589 TotalParamVariation 0.00033333
[0.46666667] Average Model
[0.5] Average Data
[0.03333333] Diff

Epoch 2590 TotalParamVariation 0.00011111
[0.51111111] Average Model
[0.5] Average Data
[-0.01111111] Diff

Epoch 2591 TotalParamVariation 0.00077778
[0.42222222] Average Model
[0.5] Average Data
[0.07777778] Diff

Epoch 2592 TotalParamVariation 0.00011111
[0.48888889] Average Model
[0.5] Average Data
[0.01111111] Diff

Epoch 2593 TotalParamVariation 0.00055556
[0.44444444] Average Model
[0.5] Average Data
[0.05555556] Diff

Epoch 2594 TotalParamVariation 0.00033333
[0.46666667] Average Model
[0.5] Average Data
[0.03333333] Diff

Epoch 2595 TotalParamVariation 0.001
[0.6] Average Model
[0.5] Average Data
[-0.1] Diff

Epoch 2596 TotalParamVariation 0.00011111
[0.51111111] Average Model
[0.5] Average Data
[-0.01111111] Diff

Epoch 2597 TotalParamVariation 0.00055556
[0.44444444] Average Model
[0.5] Average Data
[0.05555556] Diff

Epoch 2598 TotalParamVariation 0.00144444
[0.35555556

Epoch 2703 TotalParamVariation 0.00033333
[0.46666667] Average Model
[0.5] Average Data
[0.03333333] Diff

Epoch 2704 TotalParamVariation 0.00077778
[0.42222222] Average Model
[0.5] Average Data
[0.07777778] Diff

Epoch 2705 TotalParamVariation 0.00077778
[0.57777778] Average Model
[0.5] Average Data
[-0.07777778] Diff

Epoch 2706 TotalParamVariation 0.00033333
[0.53333333] Average Model
[0.5] Average Data
[-0.03333333] Diff

Epoch 2707 TotalParamVariation 0.00166667
[0.33333333] Average Model
[0.5] Average Data
[0.16666667] Diff

Epoch 2708 TotalParamVariation 0.001
[0.6] Average Model
[0.5] Average Data
[-0.1] Diff

Epoch 2709 TotalParamVariation 0.00055556
[0.44444444] Average Model
[0.5] Average Data
[0.05555556] Diff

Epoch 2710 TotalParamVariation 0.00011111
[0.51111111] Average Model
[0.5] Average Data
[-0.01111111] Diff

Epoch 2711 TotalParamVariation 0.00033333
[0.46666667] Average Model
[0.5] Average Data
[0.03333333] Diff

Epoch 2712 TotalParamVariation 0.00055556
[0.5555555

[0.5] Average Data
[0.03333333] Diff

Epoch 2820 TotalParamVariation 0.00055556
[0.44444444] Average Model
[0.5] Average Data
[0.05555556] Diff

Epoch 2821 TotalParamVariation 0.00033333
[0.53333333] Average Model
[0.5] Average Data
[-0.03333333] Diff

Epoch 2822 TotalParamVariation 0.00055556
[0.44444444] Average Model
[0.5] Average Data
[0.05555556] Diff

Epoch 2823 TotalParamVariation 0.00033333
[0.53333333] Average Model
[0.5] Average Data
[-0.03333333] Diff

Epoch 2824 TotalParamVariation 0.00077778
[0.42222222] Average Model
[0.5] Average Data
[0.07777778] Diff

Epoch 2825 TotalParamVariation 0.00033333
[0.46666667] Average Model
[0.5] Average Data
[0.03333333] Diff

Epoch 2826 TotalParamVariation 0.00055556
[0.44444444] Average Model
[0.5] Average Data
[0.05555556] Diff

Epoch 2827 TotalParamVariation 0.00011111
[0.48888889] Average Model
[0.5] Average Data
[0.01111111] Diff

Epoch 2828 TotalParamVariation 0.00033333
[0.53333333] Average Model
[0.5] Average Data
[-0.03333333] Di

Epoch 2938 TotalParamVariation 0.001
[0.4] Average Model
[0.5] Average Data
[0.1] Diff

Epoch 2939 TotalParamVariation 0.001
[0.6] Average Model
[0.5] Average Data
[-0.1] Diff

Epoch 2940 TotalParamVariation 0.00055556
[0.55555556] Average Model
[0.5] Average Data
[-0.05555556] Diff

Epoch 2941 TotalParamVariation 0.00011111
[0.48888889] Average Model
[0.5] Average Data
[0.01111111] Diff

Epoch 2942 TotalParamVariation 0.00055556
[0.55555556] Average Model
[0.5] Average Data
[-0.05555556] Diff

Epoch 2943 TotalParamVariation 0.00055556
[0.55555556] Average Model
[0.5] Average Data
[-0.05555556] Diff

Epoch 2944 TotalParamVariation 0.00077778
[0.57777778] Average Model
[0.5] Average Data
[-0.07777778] Diff

Epoch 2945 TotalParamVariation 0.00077778
[0.42222222] Average Model
[0.5] Average Data
[0.07777778] Diff

Epoch 2946 TotalParamVariation 0.00055556
[0.55555556] Average Model
[0.5] Average Data
[-0.05555556] Diff

Epoch 2947 TotalParamVariation 0.00144444
[0.64444444] Average Model


In [ ]:
s_ising.test_metropolis(simulation_len=1000, burn_in_len=100)

In [71]:
my_dict = {}
my_list = tuple([1,-1])
my_dict[my_list]=3

